In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import time

# --- 1. Configuration ---
config = {
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "BATCH_SIZE": 64,
    "NUM_EPOCHS": 10,
    "LEARNING_RATE": 0.001,
    "DATA_DIR": "/home/kami/Documents/datasets/",
    "NUM_CLASSES": 101
}

print(f"--- SCRIPT 2 (Corrected): STANDARD FINE-TUNING with ACCURACY ---")
print(f"Using device: {config['DEVICE']}")
print("-" * 60)

# --- 2. Data Loading ---
data_transforms = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
print("Loading Food101 dataset for standard training...")
train_dataset = datasets.Food101(root=config["DATA_DIR"], split='train', download=True, transform=data_transforms)
# Use shuffle=True for standard training
train_loader = DataLoader(
    train_dataset, batch_size=config["BATCH_SIZE"], shuffle=True, num_workers=4, pin_memory=True
)

# --- 3. Model Initialization ---
model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
# Freeze feature layers
for param in model.features.parameters():
    param.requires_grad = False
# Replace classifier
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096), nn.ReLU(True), nn.Dropout(0.5),
    nn.Linear(4096, 1024), nn.ReLU(True), nn.Dropout(0.5),
    nn.Linear(1024, config["NUM_CLASSES"])
).to(config["DEVICE"])
model = model.to(config["DEVICE"])

# --- 4. Standard Training Loop ---
optimizer = optim.Adam(model.classifier.parameters(), lr=config["LEARNING_RATE"])
criterion = nn.CrossEntropyLoss()
total_training_time = 0

print("--- Starting Standard Fine-Tuning Training ---")
for epoch in range(config["NUM_EPOCHS"]):
    epoch_start_time = time.time()
    model.train() # Set model to training mode

    # Variables to calculate metrics for the epoch
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['NUM_EPOCHS']}")
    for images, labels in pbar:
        images, labels = images.to(config["DEVICE"]), labels.to(config["DEVICE"])
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy stats
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    epoch_time = time.time() - epoch_start_time
    total_training_time += epoch_time

    # Calculate metrics at the end of the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(
        f"Epoch {epoch+1} completed in {epoch_time:.2f}s | "
        f"Loss: {epoch_loss:.4f} | "
        f"Accuracy: {epoch_accuracy:.2f}%"
    )

# --- 5. Final Report ---
# The metrics from the final epoch are the final results
print("-" * 60)
print(f"Finished Standard Training.")
print(f"Total Time for {config['NUM_EPOCHS']} epochs: {total_training_time:.2f} seconds")
print(f"Final Training Loss: {epoch_loss:.4f}")
print(f"Final Training Accuracy: {epoch_accuracy:.2f}%")
print("-" * 60)

--- SCRIPT 2 (Corrected): STANDARD FINE-TUNING with ACCURACY ---
Using device: cuda
------------------------------------------------------------
Loading Food101 dataset for standard training...
--- Starting Standard Fine-Tuning Training ---


Epoch 1/10: 100%|██████████| 1184/1184 [02:12<00:00,  8.92it/s, loss=3.1952]


Epoch 1 completed in 132.77s | Loss: 3.1546 | Accuracy: 24.84%


Epoch 2/10: 100%|██████████| 1184/1184 [02:15<00:00,  8.72it/s, loss=2.3959]


Epoch 2 completed in 135.72s | Loss: 2.5334 | Accuracy: 37.84%


...

Epoch 8 completed in 147.29s | Loss: 1.7955 | Accuracy: 55.62%


Epoch 9/10: 100%|██████████| 1184/1184 [02:24<00:00,  8.17it/s, loss=1.1570]


Epoch 9 completed in 144.90s | Loss: 1.7389 | Accuracy: 57.17%


Epoch 10/10: 100%|██████████| 1184/1184 [02:25<00:00,  8.15it/s, loss=2.0037]

Epoch 10 completed in 145.29s | Loss: 1.6896 | Accuracy: 58.38%
------------------------------------------------------------
Finished Standard Training.
Total Time for 10 epochs: 1425.38 seconds
Final Training Loss: 1.6896
Final Training Accuracy: 58.38%
------------------------------------------------------------
